<a href="https://colab.research.google.com/github/vicpb/word2vec-treina-word-emb-alura/blob/main/word2vec_classificador.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python -m spacy download pt_core_news_sm

✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')


In [2]:
import spacy
import numpy as np
import pandas as pd
from gensim.models import KeyedVectors

w2v_modelo_cbow = KeyedVectors.load_word2vec_format("/content/drive/MyDrive/Colab Notebooks/modelo_cbow.txt")
w2v_modelo_sg = KeyedVectors.load_word2vec_format("/content/drive/My Drive/Colab Notebooks/modelo_skipgram.txt")
artigo_treino = pd.read_csv("/content/drive/My Drive/Colab Notebooks/treino.csv")
artigo_teste = pd.read_csv("/content/drive/My Drive/Colab Notebooks/teste.csv")

In [3]:
nlp = spacy.load("pt_core_news_sm", disable=["paser", "ner", "tagger", "textcat"])

def tokenizador(texto):
    
    doc = nlp(texto)
    tokens_validos = []
    for token in doc:
        e_valido = not token.is_stop and token.is_alpha
        if e_valido:
            tokens_validos.append(token.text.lower())

    
    return  tokens_validos

texto = "Rio de Janeiro 1231231 ***** @#$ é uma cidade maravilhosa!"
tokens = tokenizador(texto)
print(tokens)

['rio', 'janeiro', 'cidade', 'maravilhosa']


In [4]:
def combinacao_de_vetores_por_soma(palavras, modelo):

    vetor_resultante = np.zeros((1,300))

    for pn in palavras:
        try:
            vetor_resultante += modelo.get_vector(pn)

        except KeyError:
            pass
                

    return vetor_resultante

vetor_texto = combinacao_de_vetores_por_soma(tokens, w2v_modelo_cbow)
print(vetor_texto.shape)
print(vetor_texto)

(1, 300)
[[-0.57215571  0.74214922 -0.35470682 -2.27220106 -0.74891359 -0.58882374
   1.97067407 -0.23945725  2.96551448 -0.5618257   1.16920155 -0.29115811
   1.13705773 -1.68892741 -0.74868731  1.20762184 -0.40211743  2.11665255
   2.01783422 -1.36546957 -0.22131144 -1.62375885 -0.44960643  0.51977116
  -2.09629428  1.84238571  0.17182408  2.88990706  2.07282785  4.34058499
  -0.94307047 -0.0152204  -0.6800046   0.02643305 -1.00582758  1.89856955
  -1.19818485  0.68375041  0.13434559 -0.07276583  0.43552956 -0.55565149
   2.35972172 -2.90523356  0.71094358 -2.02963156  1.30692664 -0.28968048
   0.65517215  0.56075428 -0.37911364  2.73371279 -0.59572794  1.76788633
  -1.1709865  -2.44322662 -0.24314688 -1.58324431 -1.49547076  0.29054198
  -0.4171164  -1.56516662 -1.42484216 -0.53194016  0.51780195 -1.56811041
   2.68951433 -2.54944938  0.5720208  -0.55467716 -0.96629435  0.23949811
   1.016515   -0.94547871  0.44745973  0.14036431  0.00453123  0.58840776
  -0.82441023  0.19396515  0.

In [5]:
def matriz_vetores(textos, modelo):
    x = len(textos)
    y = 300
    matriz = np.zeros((x,y))

    for i in range(x):
        palavras = tokenizador(textos.iloc[i])
        matriz[i] = combinacao_de_vetores_por_soma(palavras, modelo)

    return matriz

matriz_vetores_treino_cbow = matriz_vetores(artigo_treino.title, w2v_modelo_cbow)
matriz_vetores_teste_cbow = matriz_vetores(artigo_teste.title, w2v_modelo_cbow)
print(matriz_vetores_treino_cbow.shape)
print(matriz_vetores_teste_cbow.shape) 

(90000, 300)
(20513, 300)


In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

def classificador(modelo, x_treino, y_treino, x_teste, y_teste):

    RL = LogisticRegression(max_iter = 800)
    RL.fit(x_treino, y_treino)
    categorias = RL.predict(x_teste)
    resultados = classification_report(y_teste, categorias)
    print(resultados)
    
    return RL

RL_cbow = classificador(w2v_modelo_cbow,
                        matriz_vetores_treino_cbow,
                        artigo_treino.category,
                        matriz_vetores_teste_cbow,
                        artigo_teste.category)

              precision    recall  f1-score   support

     colunas       0.80      0.71      0.76      6103
   cotidiano       0.64      0.80      0.71      1698
     esporte       0.93      0.86      0.89      4663
   ilustrada       0.12      0.81      0.21       131
     mercado       0.84      0.78      0.81      5867
       mundo       0.74      0.83      0.78      2051

    accuracy                           0.79     20513
   macro avg       0.68      0.80      0.69     20513
weighted avg       0.82      0.79      0.80     20513



In [7]:
matriz_vetores_treino_sg = matriz_vetores(artigo_treino.title, w2v_modelo_sg)
matriz_vetores_teste_sg = matriz_vetores(artigo_teste.title, w2v_modelo_sg)

RL_sg = classificador(w2v_modelo_sg,
                        matriz_vetores_treino_sg,
                        artigo_treino.category,
                        matriz_vetores_teste_sg,
                        artigo_teste.category)

              precision    recall  f1-score   support

     colunas       0.81      0.71      0.76      6103
   cotidiano       0.64      0.81      0.72      1698
     esporte       0.93      0.88      0.90      4663
   ilustrada       0.14      0.89      0.25       131
     mercado       0.84      0.79      0.81      5867
       mundo       0.75      0.84      0.79      2051

    accuracy                           0.79     20513
   macro avg       0.69      0.82      0.70     20513
weighted avg       0.82      0.79      0.80     20513



In [8]:
import pickle

with open("/content/drive/My Drive/Colab Notebooks/rl_cbow.pkl", "wb") as f:
    pickle.dump(RL_cbow, f)

In [9]:
with open("/content/drive/My Drive/Colab Notebooks/rl_sg.pkl", "wb") as f:
    pickle.dump(RL_sg, f)